In [6]:
import os
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity

import dotenv
dotenv.load_dotenv()

client = OpenAI(
  api_key=os.environ.get("OPENAI_API_KEY"),
)

In [7]:
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [8]:
def get_distance(text1, text2, model="text-embedding-3-small"):
   text1 = str(text1).lower()
   text2 = str(text2).lower()
   embedding1 = get_embedding(text1, model)
   embedding2 = get_embedding(text2, model)
   return cosine_similarity([embedding1], [embedding2])[0][0]

In [9]:
def get_cosine_similarity(text1, text2, model="text-embedding-3-small"):
   embedding1 = get_embedding(text1, model)
   embedding2 = get_embedding(text2, model)
   return client.embeddings.cosine_similarity(embedding1, embedding2).data[0]

In [10]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "",
        }
    ],
    model="gpt-4o-mini",
)

str(chat_completion.choices[0].message.content)


'Hello! How can I assist you today?'

In [11]:

# go through sample.json and ask questions to gpt
import json
all_qs = json.load(open("filtered_train.json"))

# randomly sample 0 questions
import random
sample = random.sample(all_qs, 30)

print(sample)

output = []

sum_1 = 0
sum_2 = 0

for i in sample:
    # print("-"*20)
    ootb_llm_rseponse = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Rewrite this question replacing all questions with a what, but retain the meaning by specifying what entity or what person or what timeframe the \"what\" answering. Also specify the year is 2018 is needed to answer a time-based question. The Question: {i['nq_question']}",
            }
        ],
        model="gpt-4o-mini"
    )

    print(chat_completion.choices[0].message.content)


    final_answer = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"Answer the question as concisely as possible with ONLY one answer without any other text:  {chat_completion.choices[0].message.content}",
            }
        ],
        model="gpt-4o-mini"
    )


    ambig_answer_distance = 0
    disambig_answer_distance = 0
    for ans in i["nq_answer"]:
        curr_ambig_answer_distance = get_distance(ans, ootb_llm_rseponse.choices[0].message.content)
        curr_disambig_answer_distance = get_distance(ans, final_answer.choices[0].message.content)
        if curr_ambig_answer_distance > ambig_answer_distance:
            ambig_answer_distance = curr_ambig_answer_distance
        if curr_disambig_answer_distance > disambig_answer_distance:
            disambig_answer_distance = curr_disambig_answer_distance

    curr = {
        "data_id": i["nq_id"],
        "ambig_question": i["nq_question"],
        "ambig_prompt_response": ootb_llm_rseponse.choices[0].message.content,
        "disambig_question": chat_completion.choices[0].message.content,
        "disambig_prompt_response": final_answer.choices[0].message.content,
        "ground_truth": i["nq_answer"],
        "question_distance": get_distance(i["nq_question"], chat_completion.choices[0].message.content),
        "answer_distance": get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content),
        "ambig_answer_distance": ambig_answer_distance,
        "disambig_answer_distance": disambig_answer_distance
    }

    sum_1 += curr["ambig_answer_distance"]
    sum_2 += curr["disambig_answer_distance"]

    # print(f"question_distance: {get_distance(i["nq_question"], chat_completion.choices[0].message.content)}")
    # print(f"answer_distance: {get_distance(ootb_llm_rseponse.choices[0].message.content, final_answer.choices[0].message.content)}")
    # if i["nq_answer"] is list:
    # print(f"ambig_answer_distance: {get_distance(i["nq_answer"], ootb_llm_rseponse.choices[0].message.content)}")
    # print(f"disambig_answer_distance: {get_distance(i["nq_answer"], final_answer.choices[0].message.content)}")

    output.append(curr)

    # save output to json file
    json.dump(output, open("disambig_prompt_4o_mini.json", "w"))

    print(curr)

print("="*20)
print(f"Average ambig answer distance: {sum_1/len(sample)}")
print(f"Average disambig answer distance: {sum_2/len(sample)}")


[{'nq_id': '-7493939264346118144', 'nq_question': 'When does 50 shades frees come out on dvd?', 'nq_answer': ['May 8 , 2018'], 'ambig_questions': [{'question': 'When does Fifty Shades Freed come out on dvd as Blu-ray Disc?', 'answer': ['April 24, 2018']}, {'question': 'When does Fifty Shades Freed come out on dvd as 4K Ultra HD Blu-ray?', 'answer': ['May 8, 2018']}]}, {'nq_id': '6373189865465953269', 'nq_question': 'Who plays the man in the yellow hat on curious george?', 'nq_answer': ['Jeff Bennett'], 'ambig_questions': [{'question': 'Who plays the Man in the Yellow Hat on the TV series Curious George?', 'answer': ['Jeff Bennett']}, {'question': 'Who plays the Man in the Yellow Hat on the movie  Curious George?', 'answer': ['Will Ferrell']}, {'question': 'Who plays the Man in the Yellow Hat on the movie  Curious George 2: Follow That Monkey!?', 'answer': ['Jeff Bennett']}]}, {'nq_id': '-4206466522516452259', 'nq_question': 'Who came up with the idea of survival of the fittest?', 'nq_a